### Predicciones Mundial Catar 2022

**¿Que queremos responder?**

- Cual es la probabilidad de victoria de dos equipos enfrentados en el mundial (90 min)

<img src = "ima/ima_final.png">

**Descripción del problema**

- Para este problema queremos tener una aproximacion del resultado final de un partido, teniendo en cuenta el historico de los partidos, ranking de la FIFA y ultimos resultados previos entre si.


# Cargar Dataset

Importar librerias

In [1]:
import numpy as np
import pandas as pd
from itertools import repeat

Cargar archivo con historico de los partidos jugados en diferentes ligas.

In [2]:
df_orig = pd.read_csv('data/raw/results.csv')

Crear copia del df original

In [3]:
df = df_orig.copy()

Conocer datos y variables del df

In [4]:
df

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
0,1872-11-30,Scotland,England,0.0,0.0,Friendly,Glasgow,Scotland,False
1,1873-03-08,England,Scotland,4.0,2.0,Friendly,London,England,False
2,1874-03-07,Scotland,England,2.0,1.0,Friendly,Glasgow,Scotland,False
3,1875-03-06,England,Scotland,2.0,2.0,Friendly,London,England,False
4,1876-03-04,Scotland,England,3.0,0.0,Friendly,Glasgow,Scotland,False
...,...,...,...,...,...,...,...,...,...
44055,2022-09-27,Norway,Serbia,0.0,2.0,UEFA Nations League,Oslo,Norway,False
44056,2022-09-27,Sweden,Slovenia,1.0,1.0,UEFA Nations League,Stockholm,Sweden,False
44057,2022-09-27,Kosovo,Cyprus,5.0,1.0,UEFA Nations League,Pristina,Kosovo,False
44058,2022-09-27,Greece,Northern Ireland,3.0,1.0,UEFA Nations League,Athens,Greece,False


Descripción de las variables


- **date**: fecha del partido [datetime]
- **home_team**: equipo local [str]
- **away_team**: equipo visitante [str]
- **home_score**: calificacion del local [float]
- **away_score**: calificacion del visitante [float]
- **tournament**: torneo [str]
- **city**: ciudad del encuentro [str]
- **country**: pais del encuentro [str]
- **neutral**: neutralidad del campo de juego [boolean]

Corregir tipos de datos

In [5]:
df['date'] = pd.to_datetime(df['date'])

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44060 entries, 0 to 44059
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   date        44060 non-null  datetime64[ns]
 1   home_team   44060 non-null  object        
 2   away_team   44060 non-null  object        
 3   home_score  44059 non-null  float64       
 4   away_score  44059 non-null  float64       
 5   tournament  44060 non-null  object        
 6   city        44060 non-null  object        
 7   country     44060 non-null  object        
 8   neutral     44060 non-null  bool          
dtypes: bool(1), datetime64[ns](1), float64(2), object(5)
memory usage: 2.7+ MB


In [7]:
df.describe()

,home_score,away_score
count,44059.000000,44059.000000
mean,1.740507,1.178306
std,1.747525,1.394815
min,0.000000,0.000000
25%,1.000000,0.000000
50%,1.000000,1.000000
75%,2.000000,2.000000
max,31.000000,21.000000


Cargar archivo con el ranking de la FIFA

In [8]:
rank_orig = pd.read_csv('data/raw/fifa_ranking-2022-10-06.csv')

In [9]:
rank = rank_orig.copy()

Conocer datos y variables del df

In [10]:
rank

,rank,country_full,country_abrv,total_points,previous_points,rank_change,confederation,rank_date
0,1,Germany,GER,57.00,0.00,0,UEFA,1992-12-31
1,96,Syria,SYR,11.00,0.00,0,AFC,1992-12-31
2,97,Burkina Faso,BFA,11.00,0.00,0,CAF,1992-12-31
3,99,Latvia,LVA,10.00,0.00,0,UEFA,1992-12-31
4,100,Burundi,BDI,10.00,0.00,0,CAF,1992-12-31
...,...,...,...,...,...,...,...,...
63911,74,El Salvador,SLV,1330.51,1333.48,3,CONCACAF,2022-10-06
63912,75,Oman,OMA,1320.29,1323.03,0,AFC,2022-10-06
63913,76,Israel,ISR,1316.55,1316.35,0,UEFA,2022-10-06
63914,78,Georgia,GEO,1307.34,1296.46,-4,UEFA,2022-10-06


Descripción de las variables


- **rank**: ranking de la seleccion [int]
- **country_full**: nombre de la seleccion [str]
- **country_abrv**: abreviatura del nombre de la seleccion [str]
- **total_points**: total puntos actuales [float]
- **previous_points**: total puntos en corte anterior [float]
- **rank_change**: variacion del ranking con respecto al ultimo corte [int]
- **confederation**: confederacion a la que esta afiliada [str]
- **rank_date**: fecha del ranking [datetime]


# Limpieza de datos

Corregir tipos de datos

In [11]:
rank['rank_date'] = pd.to_datetime(rank['rank_date'])

In [12]:
rank.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63916 entries, 0 to 63915
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   rank             63916 non-null  int64         
 1   country_full     63916 non-null  object        
 2   country_abrv     63916 non-null  object        
 3   total_points     63916 non-null  float64       
 4   previous_points  63916 non-null  float64       
 5   rank_change      63916 non-null  int64         
 6   confederation    63916 non-null  object        
 7   rank_date        63916 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(2), int64(2), object(3)
memory usage: 3.9+ MB


In [13]:
rank.describe()

,rank,total_points,previous_points,rank_change
count,63916.000000,63916.000000,63916.000000,63916.000000
mean,100.196821,442.404780,440.999013,0.004756
std,58.060019,409.091526,409.321338,5.660809
min,1.000000,1.000000,0.000000,-92.000000
25%,50.000000,88.000000,86.000000,-1.000000
50%,100.000000,347.000000,345.000000,0.000000
75%,149.000000,641.000000,640.000000,2.000000
max,211.000000,2172.000000,2187.000000,199.000000


Filtramos los dataframe con la informacion mas reciente

In [14]:
df = df[(df['date'] >= "2018-01-01")].reset_index(drop=True)

In [15]:
rank = rank[(rank['rank_date'] >= "2018-01-01")].reset_index(drop=True)

Limpieza de datos

In [16]:
rank['country_full'] = rank['country_full'].str.replace('IR Iran', 'Iran').str.replace('Korea Republic', 'South Korea').str.replace('USA', 'United States')

# Combinar Datasets

Se realiza group by por seleccion y reset del index del df

In [17]:
rank = rank.set_index(['rank_date']).groupby(['country_full'], group_keys = False).resample('D').fillna(method = 'ffill').reset_index()

Se realiza el marge de los dos df para conocer el ranking y puntos totales de las selecciones locales y visitantes

In [18]:
df_wc = df.merge(rank[['country_full','total_points','rank','rank_date']], left_on = ['date', 'home_team'], right_on = ['rank_date', 'country_full']).drop(['rank_date', 'country_full'], axis=1)

In [19]:
df_wc = df_wc.merge(rank[['country_full','total_points','rank','rank_date']], left_on = ['date', 'away_team'], right_on = ['rank_date', 'country_full'], suffixes = ('_home', '_away') ).drop(['rank_date', 'country_full'], axis=1)

Validar datos y variables

In [20]:
df_wc

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,total_points_home,rank_home,total_points_away,rank_away
0,2018-01-18,Uganda,Namibia,0.0,1.0,African Nations Championship,Marrakech,Morocco,True,446.00,73,300.00,111
1,2018-01-19,Libya,Nigeria,0.0,1.0,African Nations Championship,Tangier,Morocco,True,389.00,89,651.00,51
2,2018-01-19,Rwanda,Equatorial Guinea,1.0,0.0,African Nations Championship,Tangier,Morocco,True,282.00,116,202.00,141
3,2018-01-20,Angola,Cameroon,1.0,0.0,African Nations Championship,Agadir,Morocco,True,200.00,142,707.00,45
4,2018-01-20,Congo,Burkina Faso,2.0,0.0,African Nations Championship,Agadir,Morocco,True,369.00,96,709.00,44
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3679,2022-09-27,Norway,Serbia,0.0,2.0,UEFA Nations League,Oslo,Norway,False,1488.57,36,1549.53,25
3680,2022-09-27,Sweden,Slovenia,1.0,1.0,UEFA Nations League,Stockholm,Sweden,False,1563.44,20,1372.48,65
3681,2022-09-27,Kosovo,Cyprus,5.0,1.0,UEFA Nations League,Pristina,Kosovo,False,1183.90,106,1180.52,108
3682,2022-09-27,Greece,Northern Ireland,3.0,1.0,UEFA Nations League,Athens,Greece,False,1441.45,49,1399.10,58


# Definir variables dependientes

Definicion de variable target tomando en cuenta resultados de los juegos

In [21]:
df_wc.loc[df_wc['home_score'] > df_wc['away_score'], 'target'] = 1 # gana el local
df_wc.loc[df_wc['home_score'] == df_wc['away_score'], 'target'] = 2 # empate 
df_wc.loc[df_wc['home_score'] < df_wc['away_score'], 'target'] = 3 # gana el visitante

Validar datos y variables

In [22]:
df_wc

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,total_points_home,rank_home,total_points_away,rank_away,target
0,2018-01-18,Uganda,Namibia,0.0,1.0,African Nations Championship,Marrakech,Morocco,True,446.00,73,300.00,111,3.0
1,2018-01-19,Libya,Nigeria,0.0,1.0,African Nations Championship,Tangier,Morocco,True,389.00,89,651.00,51,3.0
2,2018-01-19,Rwanda,Equatorial Guinea,1.0,0.0,African Nations Championship,Tangier,Morocco,True,282.00,116,202.00,141,1.0
3,2018-01-20,Angola,Cameroon,1.0,0.0,African Nations Championship,Agadir,Morocco,True,200.00,142,707.00,45,1.0
4,2018-01-20,Congo,Burkina Faso,2.0,0.0,African Nations Championship,Agadir,Morocco,True,369.00,96,709.00,44,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3679,2022-09-27,Norway,Serbia,0.0,2.0,UEFA Nations League,Oslo,Norway,False,1488.57,36,1549.53,25,3.0
3680,2022-09-27,Sweden,Slovenia,1.0,1.0,UEFA Nations League,Stockholm,Sweden,False,1563.44,20,1372.48,65,2.0
3681,2022-09-27,Kosovo,Cyprus,5.0,1.0,UEFA Nations League,Pristina,Kosovo,False,1183.90,106,1180.52,108,1.0
3682,2022-09-27,Greece,Northern Ireland,3.0,1.0,UEFA Nations League,Athens,Greece,False,1441.45,49,1399.10,58,1.0


Definir variable con la diferencia del ranking

In [23]:
df_wc['rank_dif'] = df_wc['rank_home'] - df_wc['rank_away']

Validar datos y variables

In [24]:
df_wc

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,total_points_home,rank_home,total_points_away,rank_away,target,rank_dif
0,2018-01-18,Uganda,Namibia,0.0,1.0,African Nations Championship,Marrakech,Morocco,True,446.00,73,300.00,111,3.0,-38
1,2018-01-19,Libya,Nigeria,0.0,1.0,African Nations Championship,Tangier,Morocco,True,389.00,89,651.00,51,3.0,38
2,2018-01-19,Rwanda,Equatorial Guinea,1.0,0.0,African Nations Championship,Tangier,Morocco,True,282.00,116,202.00,141,1.0,-25
3,2018-01-20,Angola,Cameroon,1.0,0.0,African Nations Championship,Agadir,Morocco,True,200.00,142,707.00,45,1.0,97
4,2018-01-20,Congo,Burkina Faso,2.0,0.0,African Nations Championship,Agadir,Morocco,True,369.00,96,709.00,44,1.0,52
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3679,2022-09-27,Norway,Serbia,0.0,2.0,UEFA Nations League,Oslo,Norway,False,1488.57,36,1549.53,25,3.0,11
3680,2022-09-27,Sweden,Slovenia,1.0,1.0,UEFA Nations League,Stockholm,Sweden,False,1563.44,20,1372.48,65,2.0,-45
3681,2022-09-27,Kosovo,Cyprus,5.0,1.0,UEFA Nations League,Pristina,Kosovo,False,1183.90,106,1180.52,108,1.0,-2
3682,2022-09-27,Greece,Northern Ireland,3.0,1.0,UEFA Nations League,Athens,Greece,False,1441.45,49,1399.10,58,1.0,-9


Organizar datafreme por equipo y fecha

In [25]:
df_wc.sort_values(['home_team', 'date'], inplace = True)

Definicion de variable racha tomando en cuenta resultados del juego anterior 

In [26]:
df_wc['racha_home_1'] = df_wc.groupby(['home_team'])['home_score'].shift(1)
df_wc['racha_away_1'] = df_wc.groupby(['away_team'])['away_score'].shift(1)

In [27]:
df_wc

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,total_points_home,rank_home,total_points_away,rank_away,target,rank_dif,racha_home_1,racha_away_1
105,2018-03-27,Afghanistan,Cambodia,2.0,1.0,AFC Asian Cup qualification,Dushanbe,Tajikistan,True,176.00,148,103.00,171,1.0,-23,NaN,NaN
727,2018-12-25,Afghanistan,Turkmenistan,0.0,2.0,Friendly,Antalya,Turkey,True,1068.00,147,1120.00,127,3.0,20,2.0,NaN
1340,2019-09-10,Afghanistan,Bangladesh,1.0,0.0,FIFA World Cup qualification,Dushanbe,Tajikistan,True,1058.00,149,922.00,182,1.0,-33,0.0,NaN
1570,2019-11-14,Afghanistan,India,1.0,1.0,FIFA World Cup qualification,Dushanbe,Tajikistan,True,1054.00,149,1201.00,106,2.0,43,1.0,NaN
1680,2019-11-19,Afghanistan,Qatar,0.0,1.0,FIFA World Cup qualification,Dushanbe,Tajikistan,True,1054.00,149,1391.00,57,3.0,92,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2138,2021-03-29,Zimbabwe,Zambia,0.0,2.0,African Cup of Nations qualification,Harare,Zimbabwe,False,1176.00,112,1265.00,90,3.0,22,0.0,0.0
2582,2021-09-03,Zimbabwe,South Africa,0.0,0.0,FIFA World Cup qualification,Harare,Zimbabwe,False,1171.88,108,1337.77,73,2.0,35,0.0,2.0
2819,2021-10-12,Zimbabwe,Ghana,0.0,1.0,FIFA World Cup qualification,Harare,Zimbabwe,False,1160.91,113,1411.30,53,3.0,60,0.0,0.0
2915,2021-11-14,Zimbabwe,Ethiopia,1.0,1.0,FIFA World Cup qualification,Harare,Zimbabwe,False,1147.34,118,1079.37,137,2.0,-19,0.0,3.0


# Dataframe final

Definir dataframe final con las variables necesarias

In [28]:
df_final = df_wc[['home_team','away_team','total_points_home','rank_home', 'total_points_away', 'rank_away', 'target', 'rank_dif','racha_home_1','racha_away_1']]

In [29]:
df_final

,home_team,away_team,total_points_home,rank_home,total_points_away,rank_away,target,rank_dif,racha_home_1,racha_away_1
105,Afghanistan,Cambodia,176.00,148,103.00,171,1.0,-23,NaN,NaN
727,Afghanistan,Turkmenistan,1068.00,147,1120.00,127,3.0,20,2.0,NaN
1340,Afghanistan,Bangladesh,1058.00,149,922.00,182,1.0,-33,0.0,NaN
1570,Afghanistan,India,1054.00,149,1201.00,106,2.0,43,1.0,NaN
1680,Afghanistan,Qatar,1054.00,149,1391.00,57,3.0,92,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...
2138,Zimbabwe,Zambia,1176.00,112,1265.00,90,3.0,22,0.0,0.0
2582,Zimbabwe,South Africa,1171.88,108,1337.77,73,2.0,35,0.0,2.0
2819,Zimbabwe,Ghana,1160.91,113,1411.30,53,3.0,60,0.0,0.0
2915,Zimbabwe,Ethiopia,1147.34,118,1079.37,137,2.0,-19,0.0,3.0


Crear variables dummies

In [30]:
df_encode_home = pd.get_dummies(df_final['home_team'], prefix = '_home')  # seleccion local

In [31]:
df_encode_home

,_home_Afghanistan,_home_Albania,_home_Algeria,_home_American Samoa,_home_Andorra,_home_Angola,_home_Anguilla,_home_Antigua and Barbuda,_home_Argentina,_home_Armenia,...,_home_United States,_home_Uruguay,_home_Uzbekistan,_home_Vanuatu,_home_Venezuela,_home_Vietnam,_home_Wales,_home_Yemen,_home_Zambia,_home_Zimbabwe
105,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
727,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1340,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1570,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1680,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2138,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2582,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2819,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2915,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [32]:
df_encode_away = pd.get_dummies(df_final['away_team'], prefix = '_away')  # seleccion visitante

In [33]:
df_encode_away

,_away_Afghanistan,_away_Albania,_away_Algeria,_away_American Samoa,_away_Andorra,_away_Angola,_away_Anguilla,_away_Antigua and Barbuda,_away_Argentina,_away_Armenia,...,_away_United States,_away_Uruguay,_away_Uzbekistan,_away_Vanuatu,_away_Venezuela,_away_Vietnam,_away_Wales,_away_Yemen,_away_Zambia,_away_Zimbabwe
105,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
727,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1340,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1570,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1680,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2138,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2582,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2819,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2915,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Concatenar dataframe final con variables dummies 

In [34]:
df_final = pd.concat([df_final,df_encode_home], axis = 1)

In [35]:
df_final = pd.concat([df_final,df_encode_away], axis = 1)

In [36]:
df_final

,home_team,away_team,total_points_home,rank_home,total_points_away,rank_away,target,rank_dif,racha_home_1,racha_away_1,...,_away_United States,_away_Uruguay,_away_Uzbekistan,_away_Vanuatu,_away_Venezuela,_away_Vietnam,_away_Wales,_away_Yemen,_away_Zambia,_away_Zimbabwe
105,Afghanistan,Cambodia,176.00,148,103.00,171,1.0,-23,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
727,Afghanistan,Turkmenistan,1068.00,147,1120.00,127,3.0,20,2.0,NaN,...,0,0,0,0,0,0,0,0,0,0
1340,Afghanistan,Bangladesh,1058.00,149,922.00,182,1.0,-33,0.0,NaN,...,0,0,0,0,0,0,0,0,0,0
1570,Afghanistan,India,1054.00,149,1201.00,106,2.0,43,1.0,NaN,...,0,0,0,0,0,0,0,0,0,0
1680,Afghanistan,Qatar,1054.00,149,1391.00,57,3.0,92,1.0,NaN,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2138,Zimbabwe,Zambia,1176.00,112,1265.00,90,3.0,22,0.0,0.0,...,0,0,0,0,0,0,0,0,1,0
2582,Zimbabwe,South Africa,1171.88,108,1337.77,73,2.0,35,0.0,2.0,...,0,0,0,0,0,0,0,0,0,0
2819,Zimbabwe,Ghana,1160.91,113,1411.30,53,3.0,60,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2915,Zimbabwe,Ethiopia,1147.34,118,1079.37,137,2.0,-19,0.0,3.0,...,0,0,0,0,0,0,0,0,0,0


Eliminar variables de selecciones locales y visitantes

In [37]:
df_final.drop(['home_team','away_team'], axis = 1, inplace = True)

Elimina valores nulos del dataframe

In [38]:
df_final.dropna(inplace = True)

In [39]:
df_final

,total_points_home,rank_home,total_points_away,rank_away,target,rank_dif,racha_home_1,racha_away_1,_home_Afghanistan,_home_Albania,...,_away_United States,_away_Uruguay,_away_Uzbekistan,_away_Vanuatu,_away_Venezuela,_away_Vietnam,_away_Wales,_away_Yemen,_away_Zambia,_away_Zimbabwe
3528,1049.77,150,966.61,171,2.0,-21,1.0,1.0,1,0,...,0,0,0,0,0,0,0,0,0,0
1889,1345.00,69,1158.00,117,1.0,-48,0.0,3.0,0,1,...,0,0,0,0,0,0,0,0,0,0
2940,1374.33,63,1038.88,153,1.0,-90,0.0,2.0,0,1,...,0,0,0,0,0,0,0,0,0,0
3463,1371.86,66,1305.92,76,3.0,-10,0.0,0.0,0,1,...,0,0,0,0,0,0,0,0,0,0
3678,1361.81,66,1379.61,63,2.0,3,0.0,2.0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2138,1176.00,112,1265.00,90,3.0,22,0.0,0.0,0,0,...,0,0,0,0,0,0,0,0,1,0
2582,1171.88,108,1337.77,73,2.0,35,0.0,2.0,0,0,...,0,0,0,0,0,0,0,0,0,0
2819,1160.91,113,1411.30,53,3.0,60,0.0,0.0,0,0,...,0,0,0,0,0,0,0,0,0,0
2915,1147.34,118,1079.37,137,2.0,-19,0.0,3.0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Configuracion de variables

Se asigna variable target del dataframe final como variable **y**

In [40]:
y = df_final[['target']]

In [41]:
y

,target
3528,2.0
1889,1.0
2940,1.0
3463,3.0
3678,2.0
...,...
2138,3.0
2582,2.0
2819,3.0
2915,2.0


Se elimina variable target del dataframe final

In [42]:
X = df_final.drop('target', axis = 1)

In [43]:
X

,total_points_home,rank_home,total_points_away,rank_away,rank_dif,racha_home_1,racha_away_1,_home_Afghanistan,_home_Albania,_home_Algeria,...,_away_United States,_away_Uruguay,_away_Uzbekistan,_away_Vanuatu,_away_Venezuela,_away_Vietnam,_away_Wales,_away_Yemen,_away_Zambia,_away_Zimbabwe
3528,1049.77,150,966.61,171,-21,1.0,1.0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1889,1345.00,69,1158.00,117,-48,0.0,3.0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2940,1374.33,63,1038.88,153,-90,0.0,2.0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3463,1371.86,66,1305.92,76,-10,0.0,0.0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3678,1361.81,66,1379.61,63,3,0.0,2.0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2138,1176.00,112,1265.00,90,22,0.0,0.0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2582,1171.88,108,1337.77,73,35,0.0,2.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2819,1160.91,113,1411.30,53,60,0.0,0.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2915,1147.34,118,1079.37,137,-19,0.0,3.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Modelo de Aprendizaje Supervisado - Random Forest Classifier

Importamos librerias de sklearn - RandomForestClassifier

In [44]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [45]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [46]:
model = RandomForestClassifier(random_state = 5, max_depth = 9, n_estimators = 200)

In [47]:
model.fit(X_train,y_train)

c:\Users\romario.silva\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.


RandomForestClassifier(max_depth=9, n_estimators=200, random_state=5)

In [48]:
y_test_pred = pd.DataFrame(model.predict(X_test), index = y_test.index, columns = ['predicciones'])

In [49]:
pred = pd.concat([y_test,y_test_pred], axis = 1)

### Comprobar efectividad del modelo

In [50]:
pred['Exito'] = (pred['target'] == pred['predicciones']).astype(int)

In [51]:
pred

,target,predicciones,Exito
3633,1.0,1.0,1
2151,1.0,1.0,1
2159,1.0,1.0,1
3499,1.0,1.0,1
1252,1.0,1.0,1
...,...,...,...
917,1.0,1.0,1
2604,1.0,1.0,1
2643,1.0,3.0,0
2605,2.0,1.0,0


#### Porcentaje de efectividad del modelo

In [52]:
print(f"Efectividad del modelo: {pred['Exito'].mean()*100}")

Efectividad del modelo: 56.060606060606055


# Prediccion de un partido

#### Ingresar equipo local y visitante

In [73]:
local = input("Ingrese equipo local: ")
visitante = input("Ingrese equipo visitante: ")

#### Identificar puntos totales, ranking y diferencia de ranking de los equipos

In [74]:
ds_rank_local = rank.query(f"country_full == '{local}'")
total_points_local = int(ds_rank_local.tail(1)["total_points"])
rank_local = int(ds_rank_local.tail(1)["rank"])

ds_rank_visitante = rank.query(f"country_full == '{visitante}'")
total_points_visitante = int(ds_rank_visitante.tail(1)["total_points"])
rank_visitante = int(ds_rank_visitante.tail(1)["rank"])

ds_df_wc_racha_home = df_wc.query(f"home_team == '{local}'")
ds_df_wc_racha_away = df_wc.query(f"away_team == '{visitante}'")

ds_df_wc_racha_home = int(ds_df_wc_racha_home.tail(1)["racha_home_1"])
ds_df_wc_racha_away = int(ds_df_wc_racha_away.tail(1)["racha_away_1"])


#### lista con las variables a evaluar

In [75]:

game_local_visitante = []
game_local_visitante.append(total_points_local)
game_local_visitante.append(rank_local)
game_local_visitante.append(total_points_visitante)
game_local_visitante.append(rank_visitante)
game_local_visitante.append(rank_local-rank_visitante)
game_local_visitante.append(ds_df_wc_racha_home)
game_local_visitante.append(ds_df_wc_racha_away)
game_local_visitante = game_local_visitante + list(repeat(0,399))

#### Dataframe con registros a evaluar

In [76]:
game_pred = pd.DataFrame( [game_local_visitante], columns = X_test.columns)
game_pred

,total_points_home,rank_home,total_points_away,rank_away,rank_dif,racha_home_1,racha_away_1,_home_Afghanistan,_home_Albania,_home_Algeria,...,_away_United States,_away_Uruguay,_away_Uzbekistan,_away_Vanuatu,_away_Venezuela,_away_Vietnam,_away_Wales,_away_Yemen,_away_Zambia,_away_Zimbabwe
0,1773,3,1759,4,-1,3,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [77]:
game_pred[f'_home_{local}'] = 1
game_pred[f'_away_{visitante}'] = 1


#### Prediccion

In [78]:
resultado = model.predict_proba(game_pred)
probabilidad_gana_local = resultado[0][0]
probabilidad_empata_local = resultado[0][1]
probabilidad_pierde_local = resultado[0][2]
print(f"{local} vs {visitante}")
print(f"Probabilidad de que gane el local: {probabilidad_gana_local*100}")
print(f"Probabilidad de que empate: {probabilidad_empata_local*100}")
print(f"Probabilidad de que pierda el local: {probabilidad_pierde_local*100}")

Argentina vs France
Probabilidad de que gane el local: 43.162201105153564
Probabilidad de que empate: 26.398133257271905
Probabilidad de que pierda el local: 30.43966563757452


In [66]:
#X.T.to_excel("equipos2.xlsx")